In [1]:
from train_models_tfrecord import single, sect1, sect2

Num GPUs Available:  1


In [2]:
net3 = sect1()
net3.initialise_data_and_model()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 126, 126, 32)      320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 63, 63, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 61, 61, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 30, 30, 64)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 57600)             0         
                                                                 
 dense (Dense)               (None, 128)               7

In [ ]:
params = {'epochs': 50,
        'tensorboard': True, 
        'cp_callback': True,
        'weights': None
        }
net3.train(params)

Epoch 1/50 - loss: 359.9530 - mean_absolute_error: 13.8127 - val_loss: 38.7271 - val_mean_absolute_error: 4.7888

In [ ]:
net3.plot()